# STRMaps

You can view the map at https://s3.amazonaws.com/filestogeaux.garyscorner.net/pub/STRMap.html FOR NOW
File will remain at s3://requesterpays.garyscorner.net/datasets/html/  (requester pays use AWS CLI [--request-payer requester])

Datasets:
* https://data.nola.gov/Housing-Land-Use-and-Blight/Map-of-Short-Term-Rental-Licenses/j5u3-2ueh

In [1]:
import pandas as pd
import folium
import numpy as np
import folium.plugins

from io import StringIO

from json import JSONEncoder
encoder = JSONEncoder().encode

from datetime import datetime

In [2]:
! curl https://data.nola.gov/api/views/en36-xvxg/rows.csv?accessType=DOWNLOAD -o DataSets/Short-Term_Rental_Permit_Applications.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.5M    0 10.5M    0     0  3763k      0 --:--:--  0:00:02 --:--:-- 3763k


In [3]:
strDf = pd.read_csv("./DataSets/Short-Term_Rental_Permit_Applications.csv")

In [4]:
#drop without location
strDf = strDf[ strDf['X'] > 0.0]

In [5]:
strDf.drop(columns=['X','Y', 'Link'],inplace=True)

In [6]:
#drop dups and withdrawn
strDf = strDf[ ~(strDf['Current Status'].isin(['Duplicate','Withdrawn'])) ]

In [7]:
strDf['Current Status'].unique()

array(['Expired', 'Denied', 'Pending', 'Revoked', 'Issued'], dtype=object)

In [8]:
#this is done in javascript now
#strDf['Link'] = strDf['Link'].map(lambda url:  f"<a target='_blank' href='{url}'>One Stop App</a>")

In [9]:
%%time

def getPoints(val):
    try:
        
        lat,lon = val[1:-1].split(', ')
        lon = float(lon)
        lat = float(lat)
        pt = (lat,lon)
        
    except Exception as e:
        print(val)
        raise(e)
        
    return pt

strDf['Location'] = strDf['Location'].map(getPoints)


CPU times: user 79 ms, sys: 4 ms, total: 82.9 ms
Wall time: 83.4 ms


In [10]:
#now done in javascript
#strDf['Address'] = strDf.apply(lambda a: f"<a target='_blank' href='http://maps.google.com/maps?q=&layer=c&cbll={a['Location'][0]},{a['Location'][1]}'>{a['Address']}</a>",axis=1)

In [11]:
strDf['Current Status'].value_counts()

Current Status
Expired    18689
Denied      8025
Pending     3468
Issued      2269
Revoked      145
Name: count, dtype: int64

In [12]:
issueDates = pd.to_datetime(strDf['Issue_Date'])
issueDatesMax = issueDates.max()

(issueDatesMax, np.count_nonzero(issueDates == issueDatesMax))

(Timestamp('2025-08-07 00:00:00'), 3)

In [13]:
statusList = list(np.unique(strDf['Current Status']))

In [14]:
colMap = {
    'Guest Occupancy Limit':'Guest Limit',
    'License Holder Name':'License Holder',
    
}

strDf.rename(columns=colMap, inplace=True)

In [15]:
#for production
outputObj = {col:strDf[col].to_list() for col in strDf.columns}

In [16]:
config = {'lastUpdated':datetime.now().strftime('%A %B %m %Y')}

In [17]:
outputWrapper = {'config':config,'data':outputObj}

In [18]:
%%time

with open('STRMapTemplate.html', 'r') as f:
    templateHTML = f.read()

templateHTML.replace('%DATETIME%', str(datetime.now()))
    
beginHTML, endHTML = templateHTML.split('DATA_ALL')

with open('STRMap.html', 'w') as f:
    f.write(beginHTML)
    f.write(encoder(outputWrapper))
    f.write(endHTML)

CPU times: user 100 ms, sys: 8.69 ms, total: 109 ms
Wall time: 109 ms


In [19]:
!ls -lh STRMap.html

-rw-r--r-- 1 g g 8.8M Aug  8 08:34 STRMap.html


In [20]:
%%time
!aws s3 cp STRMap.html s3://requesterpays.garyscorner.net/datasets/html/

upload: ./STRMap.html to s3://requesterpays.garyscorner.net/datasets/html/STRMap.html
CPU times: user 595 ms, sys: 142 ms, total: 737 ms
Wall time: 22.6 s


In [21]:
#https://s3.amazonaws.com/filestogeaux.garyscorner.net/pub/STRMap.html
!aws s3 cp s3://requesterpays.garyscorner.net/datasets/html/STRMap.html s3://filestogeaux.garyscorner.net/pub/ --acl public-read

copy: s3://requesterpays.garyscorner.net/datasets/html/STRMap.html to s3://filestogeaux.garyscorner.net/pub/STRMap.html


In [22]:
print(f"Finished {datetime.now()}")

Finished 2025-08-08 08:34:47.971487
